# **GRPO Evaluation**

## **Config**

In [1]:
import os

seed = 42
gpu_ids = "6"

CACHE_FOLDER   = "../model_cache"
METRICS_FILE   = "grpo_evaluation_metrics.csv"
DATASET_PATH   = "../datasets/cqa_test_prompt_completion.jsonl"
BATCH_SIZE     = 128

# model_name → final merged GRPO checkpoint dir
MODEL_NAMES = {
#     "Meta-Llama-3.1-8B": os.path.join("grpo_output_Meta-Llama-3.1-8B", "final_checkpoint"),
#     "Aya-23-8B":         os.path.join("grpo_output_Aya-23-8B", "final_checkpoint"),
    "SeaLLMs-v3-7B":     os.path.join("grpo_output_SeaLLMs-v3-7B", "final_checkpoint"),
#     "SEA-LION-v3-8B":    os.path.join("grpo_output_SEA-LION-v3-8B", "final_checkpoint"),
#     "Sahabat-AI-8B":   os.path.join("grpo_output_Sahabat-AI-8B", "final_checkpoint"),
}

## **Import Libraries**

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids

import gc, time, random
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)
import evaluate

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-01 01:59:16,435] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## **Utility Functions**

In [3]:
def set_global_seed(s: int = seed):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False
    set_seed(s)

def load_grpo_model_and_tokenizer(model_path: str, tokenizer_path: str, hf_token: str = None):
    # 1) reset seed
    set_global_seed()
    # 2) tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        use_auth_token=hf_token,
        local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    # 3) model (4-bit)
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_config,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token,
        local_files_only=True
    )
    model.config.use_cache = False
    model.eval()
    return tokenizer, model

def generate_grpo_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1792
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [
        text[len(prompt):]#.split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]


In [4]:
set_global_seed()

## **Load Dataset**

In [5]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 340


,prompt,completion
284,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 5 (1) Kontrak Investasi Kolektif DIRE Syariah, selain wajib memenuhi ketentuan yang diatur dalam peraturan perundang-undangan di sektor Pasar Modal yang mengatur mengenai Dana Investasi Real Estat berbentuk Kontrak Investasi Kolektif sebagaimana dimaksud dalam Pasal 4, wajib pula memuat ketentuan paling sedikit: a. Manajer Investasi dan Bank Kustodian merupakan wakil (wakiliin) yang bertindak untuk kepentingan para pemegang Unit Penyertaan DIRE Syariah berbentuk Kontrak Investasi Kolektif sebagai pihak yang diwakili (muwakil) dimana Manajer Investasi diberi wewenang untuk mengelola portofolio investasi kolektif dan Bank Kustodian diberi wewenang untuk melaksanakan Penitipan Kolektif; b. aset Real Estat, Aset Yang Berkaitan Dengan Real Estat, dan/atau kas dan setara kas tidak bertentangan dengan Prinsip Syariah di Pasar Modal; c. mekanisme pembersihan aset Real Estat, Aset Yang Berkaitan Dengan Real Estat, dan/atau kas dan setara kas dari unsur yang bertentangan dengan\n\nQuestion: Apa saja ketentuan minimal yang harus dimuat dalam Kontrak Investasi Kolektif DIRE Syariah?\nAnswer:","Mekanisme proses dijelaskan sebagai Manajer Investasi dan Bank Kustodian sebagai wakil, aset tidak bertentangan dengan Prinsip Syariah, mekanisme pembersihan aset, kata 'Syariah' pada nama, akad syariah dan skema transaksi, ringkasan akad, besarnya nisbah pembayaran, dan rencana jadwal serta tata cara pembagian hasil."
116,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\n6. \nPeminjam adalah nasabah perorangan, perusahaan, atau \nbadan yang memperoleh Penyediaan Dana dari Bank, \ntermasuk: \na. \ndebitur, untuk Penyediaan Dana berupa kredit atau \npembiayaan; \nb. \npenerbit surat berharga, pihak yang menjual surat \nberharga, \nmanajer \ninvestasi \nkontrak \ninvestasi \nkolektif, \ndan/atau \nentitas \nreferensi \n(reference entity), untuk Penyediaan Dana berupa \nsurat berharga; \nc. \npihak yang mengalihkan risiko kredit (protection \nbuyer) dan/atau entitas referensi (reference entity), \nuntuk Penyediaan Dana berupa derivatif kredit \n(credit derivatives); \nd. \npemohon (applicant), untuk Penyediaan Dana berupa \njaminan (guarantee), letter of credit, standby letter of \ncredit, atau instrumen serupa lain; \ne. \npihak tempat Bank melakukan penyertaan modal \n(investee), \nuntuk \nPenyediaan \nDana \nberupa \npenyertaan modal; \nf. \nBank atau debitur, untuk Penyediaan Dana berupa \ntagihan akseptasi; \ng. \npihak \nlawan \ntransaksi \n(counterparty), \nuntuk \nPenyediaan Dana berupa penempatan dan transaksi \nderivatif; dan/atau \nh. \npihak lain yang wajib melunasi tagihan kepada Bank.\n\nQuestion: Siapa saja yang termasuk dalam definisi Peminjam menurut dokumen ini?\nAnswer:","Dokumen ini mencakup nasabah perorangan, perusahaan, atau badan yang memperoleh Penyediaan Dana dari Bank sebagai Peminjam."
113,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 3 (1) Setiap Pihak yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 2 ayat (2), dikenakan sanksi administratif. (2) Sanksi sebagaimana dimaksud pada ayat (1) dikenakan juga kepada Pihak yang menyebabkan terjadinya pelanggaran sebagaimana dimaksud pada ayat (1). (3) Sanksi sebagaimana dimaksud pada ayat (1) dan ayat (2) dijatuhkan oleh Otoritas jasa Keuangan. (4) Sanksi administratif sebagaimana dimaksud pada ayat (1) berupa: a. peringatan tertulis; b. denda yaitu kewajiban untuk membayar sejumlah uang tertentu; c. pembatasan kegiatan usaha; d. pembekuan kegiatan usaha; e. pencabutan izin usaha; f. pem

## **Prepare Metrics**

In [6]:
if not os.path.exists(METRICS_FILE):
    pd.DataFrame(columns=[
        "model","exact_match","rouge1_f1","rouge2_f1","rougeL_f1",
        "bleu","meteor","inference_time_sec"
    ]).to_csv(METRICS_FILE, index=False)

# load evaluators
em    = evaluate.load("exact_match")
rouge = evaluate.load("rouge")
bleu  = evaluate.load("bleu")
meteor= evaluate.load("meteor")

Using the latest cached version of the module from /home/llmsosmed/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--exact_match/9d3b67e0c429cd7460b2b05aab53419b48eea369b73e1d9f185a56ca90c373d4 (last modified on Thu May  1 04:40:13 2025) since it couldn't be found locally at evaluate-metric--exact_match, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/llmsosmed/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--rouge/b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Thu May  1 04:40:15 2025) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/llmsosmed/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu May  1 04:40:19 2025) since it couldn't be found locally at 

## **Inference & Evaluation**

In [7]:
for model_key, grpo_path in MODEL_NAMES.items():
    print(f"\n▶ Evaluating GRPO {model_key}")
    tokenizer_dir = os.path.join(CACHE_FOLDER, model_key)
    tokenizer, model = load_grpo_model_and_tokenizer(grpo_path, tokenizer_dir, hf_token=os.getenv("HF_TOKEN"))

    # —– Cetak statistik bobot LoRA
    print("=== LoRA weights for", model_key, "===")
    for name, param in model.named_parameters():
        if "lora" in name:
            print(name, param.norm().item()) 
    print("="*30)
    
    # warm-up
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    preds, refs, details = [], [], []
    start = time.time()

    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=model_key):
        batch = df.iloc[i : i + BATCH_SIZE]
        prompts = batch["prompt"].tolist()
        batch_preds = generate_grpo_batch(tokenizer, model, prompts)

        preds.extend(batch_preds)
        refs.extend(batch["completion"].tolist())

        for prompt, ref, pred in zip(prompts, batch["completion"], batch_preds):
            details.append({
                "prompt":       prompt,
                "ground_truth": ref,
                model_key:      pred
            })

    inf_time = time.time() - start
    print(f"→ Inference time: {inf_time:.1f} sec")

    # compute metrics
    r_em  = em.compute(predictions=preds, references=refs)["exact_match"]
    r_rog = rouge.compute(predictions=preds, references=refs)
    r_ble = bleu.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
    r_met = meteor.compute(predictions=preds, references=refs)["meteor"]

    # append to CSV
    row = {
        "model":             model_key,
        "exact_match":       r_em,
        "rouge1_f1":         r_rog["rouge1"],
        "rouge2_f1":         r_rog["rouge2"],
        "rougeL_f1":         r_rog["rougeL"],
        "bleu":              r_ble,
        "meteor":            r_met,
        "inference_time_sec": inf_time
    }
    pd.DataFrame([row]).to_csv(METRICS_FILE, mode="a", header=False, index=False)

    # save detailed
    pd.DataFrame(details).to_json(
        f"grpo_detailed_{model_key}.jsonl",
        orient="records", lines=True
    )

    # cleanup
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()



▶ Evaluating GRPO SeaLLMs-v3-7B


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 7/7 [00:16<00:00,  2.36s/it]


=== LoRA weights for SeaLLMs-v3-7B ===
model.layers.0.self_attn.q_proj.lora_A.default.weight 1.1515291929244995
model.layers.0.self_attn.q_proj.lora_B.default.weight 0.002104587620124221
model.layers.0.self_attn.v_proj.lora_A.default.weight 1.152511715888977
model.layers.0.self_attn.v_proj.lora_B.default.weight 0.0007750959484837949
model.layers.1.self_attn.q_proj.lora_A.default.weight 1.1530708074569702
model.layers.1.self_attn.q_proj.lora_B.default.weight 0.002036202698945999
model.layers.1.self_attn.v_proj.lora_A.default.weight 1.1578419208526611
model.layers.1.self_attn.v_proj.lora_B.default.weight 0.0009062867611646652
model.layers.2.self_attn.q_proj.lora_A.default.weight 1.1503899097442627
model.layers.2.self_attn.q_proj.lora_B.default.weight 0.00204745982773602
model.layers.2.self_attn.v_proj.lora_A.default.weight 1.1541844606399536
model.layers.2.self_attn.v_proj.lora_B.default.weight 0.000833801634144038
model.layers.3.self_attn.q_proj.lora_A.default.weight 1.156881332397461
m

SeaLLMs-v3-7B: 100%|██████████| 3/3 [03:17<00:00, 65.75s/it]


→ Inference time: 197.2 sec


## **Results**

In [8]:
dfm = pd.read_csv(METRICS_FILE)
print(dfm.to_markdown(index=False))

| model         |   exact_match |   rouge1_f1 |   rouge2_f1 |   rougeL_f1 |     bleu |   meteor |   inference_time_sec |
|:--------------|--------------:|------------:|------------:|------------:|---------:|---------:|---------------------:|
| SeaLLMs-v3-7B |      0.111765 |    0.492021 |    0.416719 |    0.469391 | 0.169708 | 0.594959 |              200.588 |
| SeaLLMs-v3-7B |      0.120588 |    0.498852 |    0.423255 |    0.476282 | 0.173534 | 0.599326 |              197.237 |


In [9]:
# merge all detailed (base + GRPO) into satu DataFrame
merged = None

for key in MODEL_NAMES:
    # 1) base predictions
    base_path = f"./base_evaluation/detailed_{key}.jsonl"
    if os.path.exists(base_path):
        base_df = (
            pd.read_json(base_path, lines=True)
              .rename(columns={ key: f"pred_base_{key}" })
              [["prompt", "ground_truth", f"pred_base_{key}"]]
        )
        merged = base_df if merged is None else merged.merge(
            base_df, on=["prompt","ground_truth"], how="outer"
        )

    # 2) GRPO predictions
    grpo_path = f"grpo_detailed_{key}.jsonl"
    if os.path.exists(grpo_path):
        grpo_df = (
            pd.read_json(grpo_path, lines=True)
              .rename(columns={ key: f"pred_grpo_{key}" })
              [["prompt", "ground_truth", f"pred_grpo_{key}"]]
        )
        merged = grpo_df if merged is None else merged.merge(
            grpo_df, on=["prompt","ground_truth"], how="outer"
        )

print(f"Total rows in merged DataFrame: {len(merged)}")
display(merged.head())


Total rows in merged DataFrame: 340


,prompt,ground_truth,pred_grpo_SeaLLMs-v3-7B
0,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nKerangka Tata Kelola sebagaimana dimaksud dalam Pasal 28 ayat (1) huruf pelaksanaan fungsi kepatuhan, fungsi audit intern, dan pelaksanaan sebagaimana dimaksud pada ayat (1) huruf b, huruf d, dan huruf h sebagaimana dimaksud dalam Pasal 30 ayat (1) huruf a, serta calon Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 30 ayat (1) huruf b paling Pasal ... a. persyaratan calon anggota Direksi dan calon anggota Dewan Komisaris; tugas dan tanggung jawab Satuan Kerja Audit Intern Terintegrasi; dan pelaksanaan fungsi pengawasan oleh Dewan Komisaris; persyaratan calon anggota Dewan Pengawas Syariah; independensi tindakan Dewan Komisaris; struktur Direksi dan Dewan Komisaris; pelaksanaan fungsi manajemen risiko; Pasal 30 Pasal 31 - 12 - struktur Dewan Pengawas Syariah; pengelolaan benturan kepentingan. kebijakan remunerasi; dan b memuat paling sedikit: audit ekstern; Entitas Utama; Syariah.\n\nQuestion: Apa saja yang harus dimuat dalam kerangka Tata Kelola menurut Pasal 30?\nAnswer:","Perubahan yang diatur mencakup persyaratan calon anggota Direksi, Dewan Komisaris, Dewan Pengawas Syariah, tugas audit intern, pengawasan, manajemen risiko, dan kebijakan remunerasi.","Menurut konteks yang disediakan, tidak disebutkan apa yang harus dimuat dalam kerangka Tata Kelola menurut Pasal 30. Oleh karena itu, jawaban saya adalah ""Saya tidak tahu terkait {question}."""
1,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nKerangka Tata Kelola sebagaimana dimaksud dalam Pasal 28 ayat (1) huruf pelaksanaan fungsi kepatuhan, fungsi audit intern, dan pelaksanaan sebagaimana dimaksud pada ayat (1) huruf b, huruf d, dan huruf h sebagaimana dimaksud dalam Pasal 30 ayat (1) huruf a, serta calon Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 30 ayat (1) huruf b paling Pasal ... a. persyaratan calon anggota Direksi dan calon anggota Dewan Komisaris; tugas dan tanggung jawab Satuan Kerja Audit Intern Terintegrasi; dan pelaksanaan fungsi pengawasan oleh Dewan Komisaris; persyaratan calon anggota Dewan Pengawas Syariah; independensi tindakan Dewan Komisaris; struktur Direksi dan Dewan Komisaris; pelaksanaan fungsi manajemen risiko; Pasal 30 Pasal 31 - 12 - struktur Dewan Pengawas Syariah; pengelolaan benturan kepentingan. kebijakan remunerasi; dan b memuat paling sedikit: audit ekstern; Entitas Utama; Syariah.\n\nQuestion: Bagaimana mekanisme pelaporan audit ekstern menurut segmen ini?\nAnswer:",Saya tidak tahu terkait mekanisme pelaporan audit ekstern menurut segmen ini.,"Mekanisme pelaporan audit ekstern menurut segmen ini tidak disebutkan dalam konteks yang disediakan. Oleh karena itu, jawabannya adalah ""Saya tidak tahu terkait mekanisme pelaporan audit ekstern menurut segmen ini."""
2,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 28 \n\n(1) Lembaga Penjaminan wajib menyampaikan laporan bulanan secara lengkap dan benar kepada Otoritas Jasa Keuangan. \n\n\n(2) Ketentuan mengenai bentuk, susunan dan penyampaian laporan bulanan diatur dalam Peraturan Otoritas Jasa Keuangan mengenai Laporan Bulanan Industri Keuangan Non Bank.\n\nQuestion: Apa mekanisme pelaporan bulanan bagi Lembaga Penjaminan menurut Pasal 28?\nAnswer:",Mekanisme pelaporan dijelaskan sebagai kewajiban menyampaikan laporan bulanan lengkap dan benar kepada Otoritas Jasa Keuangan.,"Berdasarkan konteks yang disediakan, mekanisme pelaporan bulanan bagi Lembaga Penjaminan menurut Pasal 28 adalah sebagai berikut:\n\n1. Lembaga Penjaminan wajib menyampaikan laporan bulanan secara leng

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)